# Trader Behavior Insights

## Importing the libraries

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px
warnings.filterwarnings('ignore', category=RuntimeWarning)

## Importing Dataset

In [14]:
sentiment_df = pd.read_csv('/content/fear_greed_index.csv')
trader_df = pd.read_csv('/content/historical_data.csv')

In [15]:
print(sentiment_df)

       timestamp  value classification        date
0     1517463000     30           Fear  2018-02-01
1     1517549400     15   Extreme Fear  2018-02-02
2     1517635800     40           Fear  2018-02-03
3     1517722200     24   Extreme Fear  2018-02-04
4     1517808600     11   Extreme Fear  2018-02-05
...          ...    ...            ...         ...
2639  1745818200     54        Neutral  2025-04-28
2640  1745904600     60          Greed  2025-04-29
2641  1745991000     56          Greed  2025-04-30
2642  1746077400     53        Neutral  2025-05-01
2643  1746163800     67          Greed  2025-05-02

[2644 rows x 4 columns]


In [16]:
print(trader_df.head())

                                      Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
2       144.09   1150.63  BUY  02-12-2024 22:50     1002.518996       Buy   
3       142.98   1142.04  BUY  02-12-2024 22:50     1146.558564       Buy   
4         8.73     69.75  BUY  02-12-2024 22:50     1289.488521       Buy   

   Closed PnL                                   Transaction Hash  \
0         0.0  0xec09451986a1874e3a9

# 1. Data Alignment

In [17]:
# Ensure date columns are datetime and align on date (ignore time for alignment)
sentiment_df['date'] = pd.to_datetime(sentiment_df['date']).dt.date

trader_df['Timestamp IST'] = pd.to_datetime(trader_df['Timestamp IST'], dayfirst=True)

trader_df['date'] = trader_df['Timestamp IST'].dt.date

In [18]:
# Merge sentiment with trader data on date to tag sentiment per trade
merged_df = pd.merge(trader_df, sentiment_df[['date', 'value', 'classification']], on='date', how='left')

In [19]:
print(merged_df.head())

                                      Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   Size Tokens  Size USD Side       Timestamp IST  Start Position Direction  \
0       986.87   7872.16  BUY 2024-12-02 22:50:00        0.000000       Buy   
1        16.00    127.68  BUY 2024-12-02 22:50:00      986.524596       Buy   
2       144.09   1150.63  BUY 2024-12-02 22:50:00     1002.518996       Buy   
3       142.98   1142.04  BUY 2024-12-02 22:50:00     1146.558564       Buy   
4         8.73     69.75  BUY 2024-12-02 22:50:00     1289.488521       Buy   

   Closed PnL                                   Transaction Hash  \
0         0.0  0xec09451

# 2. Feature Engineering per trader

In [20]:
# For sharpe ratio
def sharpe_ratio(returns, risk_free_rate=0):
    if len(returns) < 2:
        return np.nan
    return (returns.mean() - risk_free_rate) / (returns.std() + 1e-9)

In [21]:
def compute_trader_metrics(df):
    # Filter trades per trader
    grouped = df.groupby('Account')

    metrics = []
    for trader, group in grouped:
        total_trades = len(group)
        wins = (group['Closed PnL'] > 0).sum()
        losses = (group['Closed PnL'] <= 0).sum()
        win_rate = wins / total_trades if total_trades > 0 else np.nan
        pnl = group['Closed PnL'].sum()
        # Daily returns approximation using Closed PnL grouped by day
        daily_returns = group.groupby('date')['Closed PnL'].sum().pct_change().dropna()
        sharpe = sharpe_ratio(daily_returns)
        roi = pnl / group['Size USD'].sum() if group['Size USD'].sum() != 0 else np.nan
        trade_freq = total_trades / ((group['date'].max() - group['date'].min()).days + 1)

        # Drawdown calculation
        cumulative_pnl = group.sort_values('Timestamp IST')['Closed PnL'].cumsum()
        running_max = cumulative_pnl.cummax()
        drawdown = (running_max - cumulative_pnl).max()

        metrics.append({
            'Account': trader,
            'Total Trades': total_trades,
            'Wins': wins,
            'Losses': losses,
            'Win Rate': win_rate,
            'Total PnL': pnl,
            'Sharpe Ratio': sharpe,
            'ROI': roi,
            'Trade Frequency (trades/day)': trade_freq,
            'Max Drawdown': drawdown
        })
    return pd.DataFrame(metrics)

In [22]:
trader_metrics_df = compute_trader_metrics(merged_df)

In [23]:
print(trader_metrics_df.head())

                                      Account  Total Trades  Wins  Losses  \
0  0x083384f897ee0f19899168e3b1bec365f52a9012          3818  1373    2445   
1  0x430f09841d65beb3f27765503d0f850b8bce7713          1237   599     638   
2  0x4f93fead39b70a1824f981a54d4e55b278e9f760          7584  2733    4851   
3  0x513b8629fe877bb581bf244e326a047b249c4ff1         12236  4909    7327   
4  0x72c6a4624e1dffa724e6d00d64ceae698af892a0          1430   438     992   

   Win Rate     Total PnL  Sharpe Ratio       ROI  \
0  0.359612  1.600230e+06           NaN  0.025937   
1  0.484236  4.165419e+05           NaN  0.140434   
2  0.360364  3.089759e+05           NaN  0.002383   
3  0.401193  8.404226e+05           NaN  0.001997   
4  0.306294  4.030115e+05           NaN  0.132085   

   Trade Frequency (trades/day)   Max Drawdown  
0                     24.954248  327934.104256  
1                      3.298667       0.000000  
2                     18.274699  113383.715290  
3                     

In [24]:
trader_metrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Account                       8 non-null      object 
 1   Total Trades                  8 non-null      int64  
 2   Wins                          8 non-null      int64  
 3   Losses                        8 non-null      int64  
 4   Win Rate                      8 non-null      float64
 5   Total PnL                     8 non-null      float64
 6   Sharpe Ratio                  0 non-null      float64
 7   ROI                           8 non-null      float64
 8   Trade Frequency (trades/day)  8 non-null      float64
 9   Max Drawdown                  8 non-null      float64
dtypes: float64(6), int64(3), object(1)
memory usage: 772.0+ bytes


# 3. Segmented Analysis by sentiment regimes

In [25]:
segmented_metrics = merged_df.groupby(['Account', 'classification']).agg(
    Trades=('Closed PnL', 'count'),
    Wins=('Closed PnL', lambda x: (x > 0).sum()),
    Losses=('Closed PnL', lambda x: (x <= 0).sum()),
    Total_PnL=('Closed PnL', 'sum'),
    Avg_ROI=('Closed PnL', lambda x: x.sum() / merged_df.loc[x.index, 'Size USD'].sum() if merged_df.loc[x.index, 'Size USD'].sum() != 0 else np.nan)
).reset_index()

In [26]:
segmented_metrics['Win Rate'] = segmented_metrics['Wins'] / segmented_metrics['Trades']

In [27]:
print(segmented_metrics)

                                       Account classification  Trades  Wins  \
0   0x083384f897ee0f19899168e3b1bec365f52a9012   Extreme Fear     100    37   
1   0x083384f897ee0f19899168e3b1bec365f52a9012  Extreme Greed     945    63   
2   0x083384f897ee0f19899168e3b1bec365f52a9012           Fear    1778   936   
3   0x083384f897ee0f19899168e3b1bec365f52a9012          Greed     574    92   
4   0x083384f897ee0f19899168e3b1bec365f52a9012        Neutral     421   245   
5   0x430f09841d65beb3f27765503d0f850b8bce7713   Extreme Fear      79     0   
6   0x430f09841d65beb3f27765503d0f850b8bce7713  Extreme Greed      15    15   
7   0x430f09841d65beb3f27765503d0f850b8bce7713           Fear     237    24   
8   0x430f09841d65beb3f27765503d0f850b8bce7713          Greed     772   492   
9   0x430f09841d65beb3f27765503d0f850b8bce7713        Neutral     134    68   
10  0x4f93fead39b70a1824f981a54d4e55b278e9f760   Extreme Fear     371   159   
11  0x4f93fead39b70a1824f981a54d4e55b278e9f760  Extr

# 4. Contrarian Detection

In [28]:
# Define contrarians as traders with positive ROI in Fear/extreme fear while others lose
fear_classes = ['Fear', 'Extreme Fear']

In [29]:
# Calculate mean ROI per sentiment class overall
mean_roi_by_sentiment = segmented_metrics.groupby('classification')['Avg_ROI'].mean()

In [30]:
# Traders who have positive ROI in fear classes where average ROI is negative
contrarians = segmented_metrics[
    (segmented_metrics['classification'].isin(fear_classes)) &
    (segmented_metrics['Avg_ROI'] > 0)
]

In [31]:
contrarians = contrarians[contrarians.apply(
    lambda row: row['Avg_ROI'] > mean_roi_by_sentiment.get(row['classification'], 0), axis=1
)]

In [32]:
print(contrarians)

                                       Account classification  Trades  Wins  \
0   0x083384f897ee0f19899168e3b1bec365f52a9012   Extreme Fear     100    37   
20  0x72c6a4624e1dffa724e6d00d64ceae698af892a0   Extreme Fear     221    93   
22  0x72c6a4624e1dffa724e6d00d64ceae698af892a0           Fear     431   278   

    Losses      Total_PnL   Avg_ROI  Win Rate  
0       63  124769.221441  0.082788  0.370000  
20     128  198900.561610  0.344307  0.420814  
22     153  144514.344600  0.228753  0.645012  


# 5. Trader Ranking System

In [33]:
# Simple scoring: weighted sum of metrics across sentiment classes
# Example weights - tune as needed
weights = {
    'Win Rate': 0.15,
    'Sharpe Ratio': 0.35,
    'ROI': 0.25,
    'Trade Frequency (trades/day)': 0.1
}

In [34]:
# Merge trader_metrics_df with segmented metrics averaged across sentiment classes
avg_segmented = segmented_metrics.groupby('Account').agg({
    'Win Rate': 'mean',
    'Total_PnL': 'sum',
    'Avg_ROI': 'mean',
    'Trades': 'sum'
}).reset_index()

In [35]:
# Add Sharpe and Trade Frequency from trader_metrics_df
ranking_df = pd.merge(avg_segmented, trader_metrics_df[['Account', 'Sharpe Ratio', 'Trade Frequency (trades/day)']], on='Account')

In [36]:
# Compute Score
ranking_df['Score'] = (
    ranking_df['Win Rate'] * weights['Win Rate'] +
    ranking_df['Sharpe Ratio'].fillna(0) * weights['Sharpe Ratio'] +
    ranking_df['Avg_ROI'].fillna(0) * weights['ROI'] +
    ranking_df['Trade Frequency (trades/day)'].fillna(0) * weights['Trade Frequency (trades/day)']
)

ranking_df = ranking_df.sort_values('Score', ascending=False)

In [37]:
print(ranking_df.head())

                                      Account  Win Rate     Total_PnL  \
3  0x513b8629fe877bb581bf244e326a047b249c4ff1  0.320933  8.404226e+05   
0  0x083384f897ee0f19899168e3b1bec365f52a9012  0.341065  1.600230e+06   
2  0x4f93fead39b70a1824f981a54d4e55b278e9f760  0.377622  3.089759e+05   
6  0x8381e6d82f1affd39a336e143e081ef7620a3b7f  0.275120  6.551366e+04   
5  0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4  0.587098  2.202649e+05   

    Avg_ROI  Trades  Sharpe Ratio  Trade Frequency (trades/day)     Score  
3 -0.002576   12236           NaN                     84.972222  8.544718  
0  0.032139    3818           NaN                     24.954248  2.554619  
2  0.002817    7584           NaN                     18.274699  1.884818  
6 -0.000619    1911           NaN                     12.572368  1.298350  
5  0.013132    3770           NaN                     11.223214  1.213669  


# 6. Visualization

In [38]:
# Example: ROI by sentiment class for top traders
top_traders = ranking_df.head(10)['Account']
filtered = segmented_metrics[segmented_metrics['Account'].isin(top_traders)]

fig = px.bar(filtered, x='classification', y='Avg_ROI', color='Account',
             title='Top Traders ROI by Market Sentiment',
             labels={'classification': 'Market Sentiment', 'Avg_ROI': 'Average ROI'})
fig.show()

In [39]:
# Time series of sentiment vs aggregated PnL
daily_pnl = merged_df.groupby('date')['Closed PnL'].sum().reset_index()
daily_sentiment = sentiment_df.groupby('date')['value'].mean().reset_index()

fig2 = px.line()
fig2.add_scatter(x=daily_sentiment['date'], y=daily_sentiment['value'], mode='lines', name='Sentiment Index')
fig2.add_scatter(x=daily_pnl['date'], y=daily_pnl['Closed PnL'], mode='lines', name='Aggregated PnL')
fig2.update_layout(title='Daily Market Sentiment vs Aggregated Trader PnL', yaxis_title='Value')
fig2.show()

In [40]:
# Ranking dashboard - simple bar chart
fig3 = px.bar(ranking_df.head(20), x='Account', y='Score', title='Top 20 Trader Rankings')
fig3.show()